# Gillespie's algorithm for logistic branching process where all cells are different

*Jonathan Lindström, LNU*

I am interested in modelling evolution in cancer cells. The [logistic branching process](https://projecteuclid.org/euclid.aoap/1115137984) is a stochastic branching process that follows logistic growth. I'm looking at a slightly modified version that works as follows. Consider a population of $N$ cells. Individual cells divide (into two) at a rate

$b = r - (N-1)I_b$

and die at a rate

$d = s + (N-1)I_d$

If the birth rate is ever negative, the negative excess is added to the death rate. The process will tend to hover around some population average $\hat{N}$. This is simply simulated with gillespies algorithm:

1. Calculate the birth and death rates
2. Sum birth and death rates to get a total event rate
2. Get a waiting time from the exponential distribution based on the total rate
3. Select division or death with a probability proportional to their rates.

Simmulating the process for a time t will take a time proportional to $\hat{N}$ as the number of events within a time interval depends on the number of cells. This is not a problem.

Consider a situation where every cell has a (near) unique base birth rate $r$. It is determined by the cells genome which changes upon each division (birth event), and also changes with time (simulating drug treatment). The variation in time is slow however, so we can ignore detailed effects of it changing in the simulation. The algorithm now becomes:

1. Calculate birth and death rates for each cell ($O(N)$)
2. Sum all birth and death rates to get a total event rate ($O(N)$)
2. Get waiting time
3. Select event

Note that step one grows with the number of cells. Thus simulating for a time $t$ is now $O(\hat{N}^2)$. I need to run these simulations many times to gather statistics. That is embarrasingly parallell and simple to do. But as it stands the simulations themselves are slower than what would be practical. There are two (maybe three) things I want to do with it:

1. Run multiple processes for gathering statistics on separate cores (embarassingly paralell without GPU, what about after the optimizations below?)
2. Speed up the calculation of birth and death rates (millions of cells so big vector operation) by running it on a GPU.
3. Speed up summing all the rates by also running it on the gpu, possibly as a part of calculation algorithm (more difficult).

It seems unlikely that parallelizing the rate calculation onto the CPU would provide any real-time speedup when running more than one simulation, since the other cores could be more effectively used just running more simulations. But since it is a simple repeated operation, maybe a GPU can speed it up.

### Feasibility
The whole sequential-code simulator is less than 200 lines of c++ code, designed to be flexible with the details of how the cells rates are determined. So it is not a lot of code to modify.

I have already (during the course) implemented optimization 2 above (calculating rates on GPU) using CUDA resulting in a roughly 40% speedup. However, it might be that competition for GPU resources would limit the number of simulations that can be run in parallel on one machine resulting in a net speed-loss. This would have to be investigated.

### Plan
Steps one and two below are the main focus. If time permits, also do step tree and four. If time still permits, do step five.

1. Add threading to program to run multiple simulations at once. Each simulation should have their own CUDA stream for transferring data and running kernels.
2. Investigate scaling with $\hat{N}$ and number of parallel simulations. Compare internal parallelization (with CUDA streams) with simply running more iterations of the program in parallel (is the automatic scheduling good enough?)
3. Move reduction to GPU, check effect on single simulation performance.
4. Again investigate scaling
5. (maybe) The sequential algorithm is still fully functional in the program. It is likely that small population sizes are faster running only on the CPU (no data transfer overhead). Find the size limit that optimizes performance.

I'm interested in modelling evolution in cancer cells. Specifically in a branching process model that obeys logistic growth. Such a process can be simulated using Gillespie's algorithm, but since I'm interested in a population where all cells are different, it is very inefficient. Simulating N cells for a time t is O(N^2). A large part is recalculating birth/death rates for all cells in every timestep. Since this is a large vector operation, I want to move it to the GPU for a speedup.

I also need statistics from multiple simulations. For the sequential program this is embarassingly parallel. Can the GPU be shared effectively enough that this is still true, or will the GPU segment bottleneck the sequential program when running multiple threads?

I have already implemented the GPU vector operation resulting in a decent (~40%) speedup. Now I need to add parallelization to run multiple simulations at once and test if I still get linear speedup as from the sequential program. The performance characteristics with number of parallel simulations and population size could then be measured against the pure sequential program.

There is also a costly reduction step that might also benefit from running on the GPU, this could be explored if time permits.